In [2]:
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import matplotlib.font_manager as font_manager
from sklearn.decomposition import PCA
import umap
import umap.plot

# Plotting Functions

In [3]:
def removeElements(lst, k): 
    counted = Counter(lst)
    return set([el for el in lst if counted[el] >= k])

def create_data4T_SNE(tdMatrix, labels, label, threshld, clm):
    tdMatrix =  {k.lower(): v for k, v in tdMatrix.items()}
#     print(len((set(labels.Experiment_ID))))
    labels.Experiment_ID = list(labels.Experiment_ID.str.lower())
#     print(labels.Experiment_ID[0:10])
    commonFiles = list(set(tdMatrix.keys())  & set(labels.Experiment_ID)) 
#     print(len(commonFiles))
    y = []
    for name in commonFiles:
        y.append(labels[labels[clm] == name][label].tolist()[0])
    list_of_frequent= removeElements(y, threshld)
#     print(len(list_of_frequent))
    data_X = []
    y = []
    for name in commonFiles:
        y_1 = labels[labels[clm] == name][label].tolist()[0]
        if(y_1 in list_of_frequent):
            data_X.append(tdMatrix[name])
            y.append(y_1)
    
    return np.array(data_X), y

def create_data_traintest(tdMatrix, labels, label, threshld, clm):
    tdMatrix =  {k.lower(): v for k, v in tdMatrix.items()}
    commonFiles = list(set(tdMatrix.keys())  & set(labels.Experiment_ID)) 
    y = []
# #     y_train = []
# #     y_test = []
    for name in commonFiles:
        y.append(labels[labels[clm] == name][label].tolist()[0])
    list_of_frequent= removeElements(y, threshld)
#     list_of_frequent.remove('input')
#     list_of_frequent.remove('na')
#     list_of_frequent =[ 'h3k4me1', 'h3k36me3', 'h3k9me3', 'h3k4me2', 'igg', 'pol2']
#     list_of_frequent = list(set(labels.Experiment_ID))
    data_X_train = []
    data_X_test = []
    y_train = []
    y_test = []
    
    y_train_org = []
    y_test_org = []
    
    for name in commonFiles:
#         print(name)
        y_1 = labels[labels[clm] == name][label].tolist()[0]
#         print(labels[labels[clm] == name]['data_type'].tolist()[0])
        if(y_1 in list_of_frequent):
            if(labels[labels[clm] == name]['data_type'].tolist()[0] == 'test'):
                data_X_test.append(tdMatrix[name])
                y_test.append(y_1)
                y_test_org.append(labels[labels[clm] == name]['antibody'].tolist()[0])
            else:
                data_X_train.append(tdMatrix[name])
                y_train.append(y_1)
                y_train_org.append(labels[labels[clm] == name]['antibody'].tolist()[0])
                
                
            
    return np.array(data_X_train), np.array(data_X_test), y_train, y_test, y_train_org, y_test_org



def UMAP_plot(data_X, y, label, n, nn, mtrc='cosine'):
    ump = umap.UMAP(metric=mtrc, random_state=42,  n_components=n, n_neighbors=nn)        
    ump.fit(data_X) 
    ump_data = pd.DataFrame(ump.transform(data_X)) 


    ump_data = pd.DataFrame({'X':ump_data[0],
                            'Y':ump_data[1],
                            label:y})
    ump_data.head()
    # sns.scatterplot(x="X", y="Y",
    #               data=tsne_df);
    fig =plt.figure(figsize=(20, 10))
    ax = sns.scatterplot(x="X", y="Y",
                  hue=label,
                    s= 100,
#                   palette=colors[:len(set(y))],
                  legend='full',
                sizes=(100, 900),
                  data=ump_data)
    return fig
